# Using rabbitmq-server Message Broker with Amazon AWS

### Installation

check notes on http://kronosapiens.github.io/blog/2015/04/28/rabbitmq-aws.html

Install rabbitmq-server server first:
sudo apt-get install rabbitmq-server

check if the rabbitmq server is up and runnning: sudo rabbitmqctl status

RabbitMQ comes with a “guest” user out of the box, but this user is not configured to accept external requests. This would be fine if you were planning on sending and consuming messages locally, but we are not.

The easiest way to configure RabbitMQ to accept external requests is to create a new user. Go ahead and run the following. (Feel free to chahnge the myguest username and password to anytihng you like):

sudo rabbitmqctl add_user myguest myguestpwd

sudo rabbitmqctl set_permissions -p / myguest ".*" ".*" ".*"

This will create a new user on the RabbitMQ server. We will use this username and password to configure the app-layer instance to send messages to Rabbit.

Let’s check to make sure the user was created correctly:

sudo rabbitmqctl list_users

In [ ]:
#!sudo rabbitmqctl add_user myguest myguestpwd
#!sudo rabbitmqctl set_permissions -p / myguest "." "." ".*"

In [1]:
#!pip install pika

In [1]:
import pika
import sys

In [2]:
pika.__version__

'1.0.0'

In this one we'll create a **Work Queue** that will be used to distribute time-consuming tasks among multiple workers.

The main idea behind Work Queues (aka: Task Queues) is to avoid doing a resource-intensive task immediately and having to wait for it to complete. Instead we schedule the task to be done later. We encapsulate a task as a message and send it to the queue. A worker process running in the background will pop the tasks and eventually execute the job. When you run many workers the tasks will be shared between them.


In the previous part of this tutorial we sent a message containing "Hello World!". Now we'll be sending strings that stand for complex tasks. We don't have a real-world task, like images to be resized or pdf files to be rendered, so let's fake it by just pretending we're busy - by using the time.sleep() function. We'll take the number of dots in the string as its complexity; every dot will account for one second of "work". For example, a fake task described by Hello... will take three seconds.

### Example for sending messages. They will be consumbed by workers  

In [3]:
def send_message(messagetxt):
    
    credentials = pika.PlainCredentials('myguest', 'myguestpwd')
    
    #URL='localhost'
    URL='PROD-JOB-844fd7d2202ac4da.elb.us-east-2.amazonaws.com'
    parameters =  pika.ConnectionParameters(URL, port=5672, credentials=credentials)
    connection = pika.BlockingConnection(parameters)
    channel = connection.channel()
    
    channel.queue_declare(queue='task_queue', durable=True)
    message = ' '.join(messagetxt) or "Hello World!"
    
    channel.basic_publish(exchange='',
                      routing_key='task_queue',
                      body=message,
                      properties=pika.BasicProperties(
                         delivery_mode = 2, # make message persistent
                      ))
    print(" [x] Sent %r" % message)
    connection.close()
    

By default, RabbitMQ will send each message to the next consumer, in sequence. This way of distributing messages is called **round-robin**. Try this out with two, three or more workers.

In [4]:
send_message('First.')
send_message('Second..')
send_message('Third...')
send_message('Fourth....')
send_message('Fifth.....')
send_message('Sixth......')
send_message('Seventh.......')




 [x] Sent 'F i r s t .'
 [x] Sent 'S e c o n d . .'
 [x] Sent 'T h i r d . . .'
 [x] Sent 'F o u r t h . . . .'
 [x] Sent 'F i f t h . . . . .'
 [x] Sent 'S i x t h . . . . . .'
 [x] Sent 'S e v e n t h . . . . . . .'
